In [7]:
import sqlalchemy
import pandas as pd



In [8]:
db_connection_string = 'sqlite:///./Resources/products.db'
engine  = sqlalchemy.create_engine(db_connection_string)

inspector = sqlalchemy.inspect(engine)
table_names = inspector.get_table_names()
print(table_names)

['Stock_Company_Info']


In [10]:
df = pd.read_sql_table(table_names[0], con=engine)

In [11]:
print(df.head())

   index symbol         zip              sector  fullTimeEmployees  \
0      0      A       95051          Healthcare            16400.0   
1      1     AA  15212-5858     Basic Materials            12900.0   
2      2  AAALF       65189                                 2998.0   
3      3   AABB  89169-2930     Basic Materials               42.0   
4      4  AABVF     M5H 2M5  Financial Services                1.0   

                                 longBusinessSummary         city  \
0  Agilent Technologies, Inc. provides applicatio...  Santa Clara   
1  Alcoa Corporation, together with its subsidiar...   Pittsburgh   
2  Aareal Bank AG, together with its subsidiaries...    Wiesbaden   
3  Asia Broadband, Inc., through its subsidiary, ...    Las Vegas   
4  Aberdeen International Inc., together with its...      Toronto   

          phone state        country  ... fiftyTwoWeekLow     bid  tradeable  \
0  800 227 9770    CA  United States  ...       99.810000  156.12      False   
1  4